Next, use the two log files to determine time intervals (hours) when the scheduler was unresponsive. Do this by looking for records that are the "sbatch" command from user 9204 that have return code 1 and an execution time of ~20 seconds (at least more than 15). These are commands where the scheduler timed out in responding.

3. Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import seaborn as sns
import datetime as dt
from scipy.stats import poisson, nbinom, norm
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read in both slurm files and concatenate, saving together.
files = ['data/slurm_wrapper_ce6.log', 'data/slurm_wrapper_ce5.log']

for index, value in enumerate(files):
    df = (
        pd.read_csv(value, 
                    sep = ' - ',
                    names = ['DATETIME',
                             'USER',
                             'RETRY',
                             'TIMELAPS',
                             'RETURNCODE',
                             'COMMAND'],
                    engine = 'python'))
    
    df['TIMELAPS'] = (
    df['TIMELAPS'].replace('time ', '', regex = True).astype('float64')
    )
    df['DATETIME'] = pd.to_datetime(df['DATETIME'], format='%Y-%m-%d %H:%M:%S.%f')
    
    df['DATE'] = df['DATETIME'].apply(lambda x: x.date())
    df['TIME'] = df['DATETIME'].apply(lambda x: x.time())
    
    df['UNRESPONSIVE'] = (
        np.where((df['RETURNCODE'] == 'returncode 1')\
                 & (df['TIMELAPS'] > 15),
                 True,
                 False)
    )
      
    df['SERVER'] = ''
    if index == 0:
        df['SERVER'] = 'ce6'
    else:
        df['SERVER'] = 'ce5'
    
    df['JOBID'] = (
        df['COMMAND'].\
        str.extract("\'(\d{2,})\'", expand = True)
    )
    
    df['COMMANDCOUNT'] = True
       
    if index == 0:
        all_dfs = df
    else:
        all_dfs = pd.concat(objs = [all_dfs, df], axis = 0)
all_dfs = all_dfs.set_index('DATETIME').sort_index(axis = 0, kind = 'mergesort')

In [ ]:
# Find and print various percentages of unresponsiveness based on given user commands.
overall_unresponsive_percent = (
    round(len(all_dfs['UNRESPONSIVE'].loc[all_dfs['UNRESPONSIVE'] == True])/len(all_dfs['COMMANDCOUNT']),9)
)
print("The scheduler was unresponsive for {:.2%} of all commands.\n".format(overall_unresponsive_percent))

def print_error_rate_by_command(df, list_of_commands):
    """Calculates and prints unresponsive rate by command within a given dataframe."""
    for i in list_of_commands:
        total = df['COMMANDCOUNT'].loc[df['COMMAND'].str.contains(i)]
        unresponsiveness_total = (
            df['UNRESPONSIVE'].loc[(df['UNRESPONSIVE'] == True) & (df['COMMAND'].str.contains(i))]
        )
        unresponsive_percent = (
            round(len(unresponsiveness_total)/len(total),9)
        )
        print("The scheduler was unresponsive for {:.2%} of all {} commands.\n".format(unresponsive_percent, i))

list_of_commands = ['scontrol', 'sbatch', 'sacct', 'squeue', 'scancel']
print_error_rate_by_command(all_dfs, list_of_commands)

In [ ]:
# Create consecutive periods of unresponsiveness, group them together, and find durations of those periods.
all_dfs_consecutive = all_dfs
all_dfs_consecutive['CONSECUTIVEURTIMES'] = (all_dfs_consecutive['UNRESPONSIVE'].diff(1) != 0).cumsum()

all_dfs_consecutive = all_dfs_consecutive.reset_index()

unresponsive_condition = all_dfs_consecutive['UNRESPONSIVE'] != False

other_consecutive = (
pd.DataFrame({'BEGINDATETIME' : all_dfs_consecutive.loc[unresponsive_condition == True].\
              groupby('CONSECUTIVEURTIMES')['DATETIME'].min(),
              'ENDDATETIME' : all_dfs_consecutive.loc[unresponsive_condition == True].\
              groupby('CONSECUTIVEURTIMES')['DATETIME'].max(),
              'TIMELAPS' : all_dfs_consecutive.loc[unresponsive_condition == True].\
              groupby('CONSECUTIVEURTIMES')['TIMELAPS'].max(),
              'CONSECUTIVE' : all_dfs_consecutive.loc[unresponsive_condition == True].\
              groupby('CONSECUTIVEURTIMES').size()}).reset_index(drop = True))

other_consecutive['DURATION'] = other_consecutive['ENDDATETIME'] - other_consecutive['BEGINDATETIME']
other_consecutive['DURATION'] = other_consecutive['DURATION'].dt.total_seconds()

# Where the difference between the end times and begin times is less than 15, the unresponsiveness is nonconsecutive.
other_consecutive['DURATION'] = (
    other_consecutive['DURATION'].loc[other_consecutive['DURATION'] > 15]
)

other_consecutive = other_consecutive['DURATION'].dropna()
round(other_consecutive.describe(), 2)

In [ ]:
all_dfs_nonconsecutive = all_dfs
all_dfs_nonconsecutive['NCURTIMES'] = (
    np.where((all_dfs_nonconsecutive['UNRESPONSIVE'].diff(1) == 0)\
             & (all_dfs_nonconsecutive['UNRESPONSIVE'] == True),
             True,
             False))

all_dfs_nonconsecutive = (
    all_dfs_nonconsecutive['TIMELAPS'].loc[all_dfs_nonconsecutive['NCURTIMES'] == True]
)

all_dfs_nonconsecutive_by_minute = pd.DataFrame(
    all_dfs_nonconsecutive.groupby(pd.Grouper(freq = '1min')).max())

all_dfs_nonconsecutive_by_minute = all_dfs_nonconsecutive_by_minute.dropna()
round(all_dfs_nonconsecutive_by_minute.describe(), 2)

In [ ]:
timeouts = pd.DataFrame(np.sort(all_dfs['TIMELAPS'].loc[all_dfs['UNRESPONSIVE'] == True]))
timeouts = timeouts.dropna()
print(round(timeouts.describe(), 2))

In [ ]:
timeout_times = list(np.sort(timeouts[0]))
consecutive_times = list(np.sort(other_consecutive))
nonconsecutive_times = list(np.sort(all_dfs_nonconsecutive_by_minute['TIMELAPS']))

fig = plt.figure(figsize = (15, 5),dpi = 400)

list_of_times = [timeout_times, consecutive_times, nonconsecutive_times]
colors = ['red', 'blue', 'gold']
for i, ls in enumerate(list_of_times):
    sns.ecdfplot(ls, color = colors[i], linewidth = 1.5)

plt.legend(labels = ['Each Timeout', 'Consecutive Periods', 'Approx. Non-Consecutive Periods'], loc = 'upper left')
plt.xlabel('Approximate Duration in Seconds of Unresponsiveness')
plt.ylabel('Proportion')
plt.title('ECDFs of Approximate Duration in Seconds of Unresponsiveness')
plt.xlim(-1, 80)
plt.savefig('images/ECDFs.png', dpi = 300, transparent = True);

In [ ]:
# Group data by sum of unresponsive counts per hour.
relevant_columns = ['UNRESPONSIVE', 'COMMANDCOUNT']

for index, value in enumerate(relevant_columns):
    df = (
        all_dfs.groupby(pd.Grouper(freq = 'H'))[value].\
        sum().\
        reset_index().\
        rename(columns = {'sum':f'{value}'}))
    if index == 0:
        hour_dfs = df
    else:
        hour_dfs = hour_dfs.merge(df, on = 'DATETIME')

In [ ]:
responsiveness_by_day = pd.DataFrame()
responsiveness_by_week = pd.DataFrame()
responsiveness_by_month = pd.DataFrame()

list_of_samples = ['D','W','M']

list_of_dfs = [responsiveness_by_day, 
               responsiveness_by_week, 
               responsiveness_by_month]

new_list_of_dfs = []

for index, i in enumerate(list_of_dfs):
    i = (
        pd.DataFrame(hour_dfs.resample(list_of_samples[index], 
                                       on = 'DATETIME',
                                       origin = 'start')['UNRESPONSIVE', 
                                                         'COMMANDCOUNT'].sum())
    )
    for j in ['UNRESPONSIVE','COMMANDCOUNT']:
        i[j + 'MINMAX'] = (
            (i[j] - i[j].min())/(i[j].max() - i[j].min())
    )
    new_list_of_dfs.append(i)

responsiveness_by_day = new_list_of_dfs[0]
responsiveness_by_week = new_list_of_dfs[1]
responsiveness_by_month = new_list_of_dfs[2]

list_of_dfs = [responsiveness_by_day, 
               responsiveness_by_week, 
               responsiveness_by_month]

In [ ]:
# Plot line graphs of unresponsive counts over time.
# Possibly reduce xtick number.
facecolor = 'white'
fig = plt.figure(figsize = (15, 12), 
                 dpi = 400)
axes = fig.subplots(nrows = 3)
plt.subplots_adjust(hspace = 0.5)

x_labels = ['Day', 
            'Week', 
            'Month']
y_labels = ['Relative Values Per Day', 
            'Relative Values Per Week', 
            'Relative Values Per Month']

for index, var in enumerate(list_of_dfs):
    y1 = var['UNRESPONSIVEMINMAX']
    y2 = var['COMMANDCOUNTMINMAX']
    x = var.index
    axes[index].plot(x, y1, linewidth = 2, color = 'red')
    axes[index].plot(x, y2, linewidth = 2, color = 'dodgerblue', alpha = 0.5)
    axes[index].set_xlabel(x_labels[index])
    axes[index].set_ylabel(y_labels[index])
    axes[index].set_title(f"Unresponsiveness and User Commands by {x_labels[index]}")
    axes[index].xaxis.set_major_formatter(mdates.DateFormatter('%D'))
    axes[index].legend(['Unresponsive Counts',
                        'User Command Counts'], 
                       loc = 'upper right', 
                       bbox_to_anchor = (1, 1))
    plt.setp(axes[index].get_xticklabels(), 
             rotation = 90)
    axes[index].set_ylim(-0.05,1.05)
    if index != 2:
        axes[index].set_xticks(responsiveness_by_week.index)
        axes[index].set_xlim(responsiveness_by_week.index.min(),responsiveness_by_week.index.max())
    else:
        axes[index].set_xticks(responsiveness_by_month.index)
        axes[index].set_xlim(responsiveness_by_month.index.min(),responsiveness_by_month.index.max())
        
plt.savefig('images/TimeSeries.png', dpi = 300, transparent = True);

In [ ]:
# Try to limit models to days where system was actually being used.
scatter_responsiveness_by_day = responsiveness_by_day.loc[responsiveness_by_day['COMMANDCOUNT'] > 0]

# Poisson model using log of additional user commands per day as explanatory variable.
full_model_exog = sm.add_constant(np.log(scatter_responsiveness_by_day['COMMANDCOUNT']))

poisson_model = sm.GLM(endog = scatter_responsiveness_by_day['UNRESPONSIVE'],
                       exog = full_model_exog,
                       family = sm.families.Poisson()).fit()

# Negative binomial model using log of additional user commands per day as explanatory variable.
nb_model = sm.GLM(endog = scatter_responsiveness_by_day['UNRESPONSIVE'],
                  exog = full_model_exog,
                  family = sm.families.NegativeBinomial()).fit()

# Run Negative Binomial Auxiliary model.
predicted_aux = poisson_model.predict(full_model_exog)

aux_model = sm.GLM(endog = ((scatter_responsiveness_by_day['UNRESPONSIVE'] - predicted_aux)**2 - predicted_aux)\
                   / predicted_aux,
                   exog = predicted_aux,
                   family = sm.families.Gaussian()).fit()

nb_aux_model = sm.GLM(endog = scatter_responsiveness_by_day['UNRESPONSIVE'],
                      exog = full_model_exog,
                      family = sm.families.NegativeBinomial(alpha = aux_model.params['x1'])).fit()

print('Poisson Model' + '\n')
print(poisson_model.summary())
print('\n\n' + 'Negative Binomial Model' + '\n')
print(nb_model.summary())
print('\n\n' + 'Negative Binomial Auxiliary Model' + '\n')
print(nb_aux_model.summary())

In [ ]:
# Test distribution of counts based on 10,000 user commands per day.
# Consider coloring bars based on quantiles.
test_number = 10000
graph_exog = sm.add_constant(np.array([np.log(test_number)]), has_constant = 'add')

# Predict mu value based on Poisson model.
poisson_predicted_mu = poisson_model.predict(graph_exog)

# Create x-range.
x_1 = np.arange(start = 0, stop = 500, step = 1)

# Create y-range for Poisson model.
y_1 = poisson.pmf(x_1, poisson_predicted_mu)

# Plot figure.
plt.figure(figsize = (15, 6), dpi = 500)
plt.bar(x_1, y_1, color = 'orange')
plt.title(f'Poisson Projected Probabilities of UC Given {test_number:,.0f} User Commands on One Day')
plt.ylabel('Poisson Probability')
plt.xlabel('Poisson Unresponsive Counts')
plt.xlim(150, 350)

plt.savefig('images/PoissonDist.png', dpi = 300, transparent = True);

In [ ]:
# Predict mu value based on Negative Binomial model.
# Consider plt.annotate or similar to show distribution points or quantiles.
alpha = 1
nb_predicted_mu = nb_model.predict(graph_exog)

# Create x-range.
x_2 = np.arange(start = 0, stop = 1500, step = 1)

# Create y-range for NB model.
nb_sigma = np.sqrt(nb_predicted_mu + alpha*nb_predicted_mu**2)
nb_p = nb_predicted_mu / nb_sigma**2
nb_n = nb_predicted_mu**2 / (nb_sigma**2 - nb_predicted_mu)

y_2 = nbinom.cdf(x_2, p = nb_p, n = nb_n)

# Plot figure.
plt.figure(figsize = (15, 8), dpi = 500)
plt.plot(x_2, y_2, color = 'blue')
plt.title(f'NB Projected CDF of UC Given {test_number:,.0f} User Commands on One Day')
plt.ylabel('NB Probability')
plt.xlabel('NB Unresponsive Counts')
plt.xlim(0, 1500)
plt.ylim(0, 1)

plt.savefig('images/NBDist.png', dpi = 300, transparent = True);

In [ ]:
# Predict mu value based on Negative Binomial Auxiliary model.
# Consider plt.annotate or similar to show distribution points or quantiles.
alpha = aux_model.params['x1']
nb_aux_predicted_mu = nb_aux_model.predict(graph_exog)

# Create x-range.
x_3 = np.arange(start = 0, stop = 1500, step = 1)

# Create y-range for NB model.
nb_aux_sigma = np.sqrt(nb_aux_predicted_mu + alpha*nb_aux_predicted_mu**2)
nb_aux_p = nb_aux_predicted_mu / nb_aux_sigma**2
nb_aux_n = nb_aux_predicted_mu**2 / (nb_aux_sigma**2 - nb_aux_predicted_mu)

y_3 = nbinom.cdf(x_3, p = nb_aux_p, n = nb_aux_n)

# Plot figure.
plt.figure(figsize = (15, 8), dpi = 500)
plt.plot(x_3, y_3, color = 'red')
plt.title(f'NB Auxiliary Projected CDF of UC Given {test_number:,.0f} User Commands on One Day')
plt.ylabel('NB Auxiliary Probability')
plt.xlabel('NB Auxiliary Unresponsive Counts')
plt.xlim(0, 1500)
plt.ylim(0, 1)

plt.savefig('images/NBAuxDist.png', dpi = 300, transparent = True);

In [ ]:
# Plot scatterplot of user commands and unresponsive counts.
# Also plot regression lines on top of scatter plot to see fit visually.

# Set plot fundamentals.
fig, ax = plt.subplots(figsize = (12, 8), dpi = 300)
x = scatter_responsiveness_by_day['COMMANDCOUNT']
y = scatter_responsiveness_by_day['UNRESPONSIVE']

ax.scatter(x, y, color = 'k', alpha = 0.5)

# Create x prediction range for models.
x_predict_range = np.linspace(start = np.log(scatter_responsiveness_by_day['COMMANDCOUNT']).min(),
                              stop = np.log(scatter_responsiveness_by_day['COMMANDCOUNT']).max(), 
                              num = len(scatter_responsiveness_by_day['COMMANDCOUNT']))

# Generate y values from model.
y_1 = poisson_model.predict(sm.add_constant(x_predict_range))
y_2 = nb_model.predict(sm.add_constant(x_predict_range))
y_3 = nb_aux_model.predict(sm.add_constant(x_predict_range))

# Plot regression lines.
poisson_line, = ax.plot(np.exp(x_predict_range),y_1, color = 'orange')
nb_line, = ax.plot(np.exp(x_predict_range),y_2, color = 'blue')
nb_aux_line, = ax.plot(np.exp(x_predict_range),y_3, color = 'red')

# Create legend for each line.
line_legend = ax.legend([poisson_line, nb_line, nb_aux_line],
                        ['Poisson','Negative Binomial', 'Negative Binomial Aux'],
                        loc = 'upper left')
ax.add_artist(line_legend)

# Boilerplate formatting for MPL.
ax.set_xticks(np.arange(0,160000,20000))
ax.set_yticks(np.arange(0,60000,10000))
ax.get_xaxis().set_major_formatter(mtick.FuncFormatter(lambda x, p: format(int(x), ',')))
ax.get_yaxis().set_major_formatter(mtick.FuncFormatter(lambda x, p: format(int(x), ',')))
plt.xlabel('User Command Submissions')
plt.ylabel('Unresponsive Counts')
plt.xlim(-1000, scatter_responsiveness_by_day['COMMANDCOUNT'].max() + 1000)
plt.ylim(-1000, scatter_responsiveness_by_day['UNRESPONSIVE'].max() + 1000)
plt.title('Scatter Plot Comparing User Command Submissions and Unresponsive Counts', fontsize = 10)

plt.savefig('images/ScatterRegression.png', dpi = 300, transparent = True);

In [ ]:
def get_rqr_poisson(observed_endog, mu):
    """Obtain randomized quantile residuals for a Poisson model."""
    return norm.ppf(poisson.cdf(mu = mu, k = observed_endog - 1)\
                    + np.random.uniform(0, 1, size = len(observed_endog))\
                    * poisson.pmf(mu = mu, k = observed_endog))

def get_rqr_nb(observed_endog, n, p):
    """Obtain randomized quantile residuals for a Negative Binomial model."""
    return norm.ppf(nbinom.cdf(n = n, p = p, k = observed_endog - 1)\
                    + np.random.uniform(0, 1, size = len(observed_endog))\
                    * nbinom.pmf(n = n, p = p, k = observed_endog))

def qq_plot(data, name, rqr = False):
    """Generates qqplot from data with given name."""
    data = sorted(data)
    
    # Create different x-ranges based upon whether RQRs will be used for the plot.
    if rqr == False:
        theoretical = np.linspace(start = 0,
                                  stop = 1,
                                  num = len(data))
    else:
        theoretical = norm.ppf(np.linspace(start = 0,
                                           stop = 1,
                                           num = len(data) + 2)[1:-1])

    plt.figure(figsize = (8, 6), dpi = 200)
    
    plt.scatter(theoretical, data, color = 'blue')
    
    if rqr == True:
        xmin, xmax = plt.xlim()
    else:
        xmin = 0
        xmax = 1
        plt.xlim(xmin, xmax)
        plt.ylim(xmin, xmax)
        
    plt.plot([xmin, xmax], [xmin, xmax], color = 'black')
    plt.rcParams["font.size"] = 8
    plt.xlabel('Theoretical Distribution')
    plt.ylabel('Observed Distribution')
    
    if rqr == True:
        plt.title(f'QQ Plot of {name} Model')
    else:
        plt.title(f'QQ Plot of {name} Model')
    
    plt.savefig(f'images/{name}QQPlot.png', dpi = 300, transparent = True);
              
    plt.show()
    
number_of_simulations = 1000      

def generate_qq_plot_for_model(sims, model, exog, observed_endog, rqr = False):

    mu = model.predict(exog)
    
    if type(model.family) == sm.genmod.families.family.NegativeBinomial:
        sigma = np.sqrt(mu + alpha*mu**2)
        p = mu / sigma**2
        n = mu**2 / (sigma**2 - mu)
        simulation = nbinom.rvs(p = p, n = n, size = (sims, len(mu)))
        name = "Negative Binomial"
        if rqr == True:
            qq_plot(get_rqr_nb(observed_endog, n, p), name, rqr = True)
        else:
            qq = sorted((simulation < np.array([observed_endog for _ in range(sims)])).\
                sum(axis = 0) / sims)
            qq_plot(qq, name)
        
    elif type(model.family) == sm.genmod.families.family.Poisson:
        simulation = poisson.rvs(mu = mu, size = (sims, len(mu)))
        name = "Poisson"
        if rqr == True:
            qq_plot(get_rqr_poisson(observed_endog, mu), name, rqr = True)
        else:
            qq = sorted((simulation < np.array([observed_endog for _ in range(sims)])).\
                sum(axis = 0) / sims)
            qq_plot(qq, name)


# Call models with run-of-the-mill QQ plots.            
models = [poisson_model, nb_model, nb_aux_model]
for index, model in enumerate(models):
    alpha = 1
    if index > 1:
        alpha = aux_model.params['x1']
    generate_qq_plot_for_model(number_of_simulations, 
                               model, 
                               full_model_exog, 
                               scatter_responsiveness_by_day['UNRESPONSIVE'])

In [ ]:
# Plot QQ Plots adjusted for randomized quantile residuals for each model.
for index, model in enumerate(models):
    alpha = 1
    if index > 1:
        alpha = aux_model.params['x1']
    generate_qq_plot_for_model(number_of_simulations, 
                               model, 
                               full_model_exog, 
                               scatter_responsiveness_by_day['UNRESPONSIVE'], 
                               rqr = True)